In [1]:
import numpy as np
import pandas as pd
df = pd.read_pickle( 'Project-3_NYC_311_Calls.pkl')


# We make the index as a proper DatetimeIndex, and then delete the Created Date column

In [2]:
df = df.set_index(pd.DatetimeIndex(df['Created Date']))
del df['Created Date']

In [3]:
df.head()

,Unique Key,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,City,Resolution Description,Borough,Open Data Channel Type
Created Date,,,,,,,,,,,
2011-04-06 00:00:00,20184537,HPD,Department of Housing Preservation and Develop...,HEATING,HEAT,RESIDENTIAL BUILDING,10002.0,NEW YORK,More than one complaint was received for this ...,MANHATTAN,UNKNOWN
2011-04-06 00:00:00,20184538,HPD,Department of Housing Preservation and Develop...,GENERAL CONSTRUCTION,WINDOWS,RESIDENTIAL BUILDING,11236.0,BROOKLYN,The Department of Housing Preservation and Dev...,BROOKLYN,UNKNOWN
2011-04-06 00:00:00,20184539,HPD,Department of Housing Preservation and Develop...,PAINT - PLASTER,WALLS,RESIDENTIAL BUILDING,10460.0,BRONX,The Department of Housing Preservation and Dev...,BRONX,UNKNOWN
2022-07-08 11:14:43,54732265,DSNY,Department of Sanitation,Dirty Condition,Trash,Sidewalk,10467.0,BRONX,The Department of Sanitation investigated this...,BRONX,PHONE
2011-04-06 00:00:00,20184540,HPD,Department of Housing Preservation and Develop...,NONCONST,VERMIN,RESIDENTIAL BUILDING,10460.0,BRONX,The Department of Housing Preservation and Dev...,BRONX,UNKNOWN


# Question 1

In [4]:
## We have to make the unique key column as that there is no missing values, 
## and also we have to make sure this the data we have in 2022.
dailycomplaints = df['Unique Key'].resample('D').count()
dailycomplaints2022 = dailycomplaints['2022']
averagedailycomplaints_2022 = dailycomplaints2022.mean()
print("The average number of daily complaints received in 2022 is",averagedailycomplaints_2022)


The average number of daily complaints received in 2022 is 8684.320547945206


# Question 2

In [5]:
## This is how we get the maximum number, simply just using the function idxmax()
dailycomplaints.idxmax()

Timestamp('2020-08-04 00:00:00')

# Question 3

In [6]:
## Since we got the maximum number date is 2020-8-4, so we put into the data, and check the numbers of values
maximumcallsdate = '2020-08-04'
maximumcallsdate = pd.Timestamp(maximumcallsdate )
monthname = 'Feb'
tempdf = df[df.index.date == maximumcallsdate.date()]
tempdf.head()

tempdf['Complaint Type'].value_counts()

Complaint Type
Damaged Tree                           14863
Noise - Residential                      982
Request Large Bulky Item Collection      909
Street Light Condition                   617
Overgrown Tree/Branches                  609
                                       ...  
Bus Stop Shelter Placement                 1
Unsanitary Pigeon Condition                1
Public Payphone Complaint                  1
For Hire Vehicle Report                    1
Bridge Condition                           1
Name: count, Length: 125, dtype: int64

## As we can see damaged tree has the most number which is 14853, so it is the most important complaint type.

# Question 4

In [7]:
monthly_complaints = df.resample('M').size()
# Identify the month with the fewest complaints
fewest_complaints_month = monthly_complaints.idxmin()
# Get the abbreviated month name
month_name = fewest_complaints_month.strftime('%b')
print("The month with the fewest complaints is:",monthname)


The month with the fewest complaints is: Feb


# Question 5

In [11]:
from statsmodels.tsa.seasonal import seasonal_decompose
# we have to make the time series to a dily frequency
daily_complaints = df['Unique Key'].resample('D').count()
# Perform ETS Decomposition
result = seasonal_decompose(daily_complaints.dropna(), model='additive') 
seasonal_component = result.seasonal
# Extract and analyze the seasonal component, specifically for the date 2020-12-25
round(seasonal_component['2020-12-25'])
print('the value of the seasonal component on 2020-12-25 is', round(seasonal_component['2020-12-25']))

the value of the seasonal component on 2020-12-25 is 183


# Question 6

In [12]:
# We need to calcualte the autocorrelation of the number of daily celss with the number of calls just simply using the lag=1.
dailycomplaints.autocorr(lag=1)


0.7517059728398577

# Question 7

In [16]:
from prophet import Prophet
from sklearn.metrics import mean_squared_error
from math import sqrt
data = daily_complaints.reset_index()
data.columns = ['ds','y']
train = data.iloc[:-90]
test = data.iloc[-90:]
# We have to use the prophet and reset the index.
model = Prophet()
model.fit(train)
# Making the train model and set up the periods is 90
future = model.make_future_dataframe(periods=90)
forecast = model.predict(future)
y_true = test['y'].values
y_pred = forecast.iloc[-90:]['yhat'].values
# Calculating the rmse
rmse = sqrt(mean_squared_error(y_true, y_pred))
rmse

01:46:53 - cmdstanpy - INFO - Chain [1] start processing
01:46:54 - cmdstanpy - INFO - Chain [1] done processing


1231.5587623794388